This script is used to create WRF domain boundaries from user input.  It's supposed to help make it a little more simple to create nested domains in WRF without having to think too hard about the specific variables in the namelist.input file.

In [2]:
# import functions
# OS interaction and time
import os
import sys
import cftime
import datetime
import time
import glob
import dask
import dask.bag as db
import calendar
import importlib

# math and data
import math
import numpy as np
import netCDF4 as nc
import xarray as xr
import scipy as sp
import scipy.linalg
from scipy.signal import detrend
import pandas as pd
import pickle as pickle
from sklearn import linear_model
import matplotlib.patches as mpatches
from shapely.geometry.polygon import LinearRing
import statsmodels.stats.multitest as multitest

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import matplotlib.image as mpimg
from matplotlib.colors import TwoSlopeNorm
import matplotlib.cm as cm

from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.axes_divider import HBoxDivider
import mpl_toolkits.axes_grid1.axes_size as Size
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point

# random
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# paths to various directories
rda_era5_path = '/glade/campaign/collections/rda/data/ds633.0/'  # base path to ERA5 data on derecho
my_era5_path = '/glade/u/home/zcleveland/scratch/ERA5/'  # path to subset data
misc_data_path = '/glade/u/home/zcleveland/scratch/misc_data/'  # path to misc data
plot_out_path = '/glade/u/home/zcleveland/NAM_soil-moisture/ERA5_analysis/plots/'  # path to generated plots
scripts_main_path = '/glade/u/home/zcleveland/NAM_soil-moisture/scripts_main/'  # path to my dicts, lists, and functions

# import variable lists and dictionaries
if scripts_main_path not in sys.path:
    sys.path.insert(0, scripts_main_path)  # path to file containing these lists/dicts
if 'get_var_data' in sys.modules:
    importlib.reload(sys.modules['get_var_data'])
if 'my_functions' in sys.modules:
    importlib.reload(sys.modules['my_functions'])
if 'my_dictionaries' in sys.modules:
    importlib.reload(sys.modules['my_dictionaries'])

# import common functions that I've created
from get_var_data import get_var_data, get_var_files, open_var_data, subset_var_data, time_to_year_month_avg, time_to_year_month_sum, time_to_year_month
from my_functions import month_num_to_name, ensure_var_list

# import lists and dictionaries
from my_dictionaries import (
sfc_instan_list, sfc_accumu_list, pl_var_list, derived_var_list, invar_var_list,
NAM_var_list, region_avg_list, flux_var_list, vector_var_list, misc_var_list,
var_dict, var_units, region_avg_dict, region_avg_coords, region_colors_dict
)

In [ ]:
class WRF_Namelist:
    def __init__(self, start_date, run_time, domain_coords, horizontal_resolution, 
                 history_interval='60-00', frames_per_outfile=1000):
        """
        Initialize the WRF_Namelist object with the given parameters.

        Parameters:
        start_date (str): Start date in 'YYYY-MM-DD-HH-mm-ss' format.
        run_time (str): Run time in 'DD-HH-mm-ss' format.
        domain_coords (dict): Domain coordinates as {'north': xx, 'south': xx, 'west': xx, 'east': xx}.
        horizontal_resolution (float): Horizontal resolution in km.
        history_interval (str): Interval at which history files are written, default '60-00'.
        frames_per_outfile (float): Number of frames per output file, default 1000.
        """
        self.start_date = start_date
        self.run_time = run_time
        self.domain_coords = domain_coords
        self.horizontal_resolution = horizontal_resolution
        self.history_interval = history_interval
        self.frames_per_outfile = frames_per_outfile

        # Parse the input dates and calculate the end date
        self.start_date_dt = datetime.datetime.strptime(self.start_date, '%Y-%m-%d-%H-%M-%S')
        self.run_time_dt = datetime.timedelta(days=int(run_time.split('-')[0]),
                                              hours=int(run_time.split('-')[1]),
                                              minutes=int(run_time.split('-')[2]),
                                              seconds=int(run_time.split('-')[3]))
        self.end_date_dt = self.start_date_dt + self.run_time_dt

        # Split the history interval
        self.history_interval_m, self.history_interval_s = map(int, self.history_interval.split('-'))

        # Calculate domain boundaries and resolution
        self.calculate_domain_boundaries()

    def calculate_domain_boundaries(self):
        """
        Calculate the domain boundaries and resolutions.
        """
        self.dx = self.dy = self.horizontal_resolution * 1000  # Convert km to meters

        # Assume a simple grid setup for the example
        self.west_east = int((self.domain_coords['east'] - self.domain_coords['west']) * 111000 / self.dx)
        self.south_north = int((self.domain_coords['north'] - self.domain_coords['south']) * 111000 / self.dy)

    def generate_namelist(self, filename='namelist.input'):
        """
        Generate the namelist.input file.

        Parameters:
        filename (str): The name of the file to write the namelist to.
        """
        with open(filename, 'w') as f:
            f.write("&time_control\n")
            f.write(f" run_days                            = {self.run_time_dt.days},\n")
            f.write(f" run_hours                           = {self.run_time_dt.seconds // 3600},\n")
            f.write(f" run_minutes                         = {(self.run_time_dt.seconds // 60) % 60},\n")
            f.write(f" run_seconds                         = {self.run_time_dt.seconds % 60},\n")
            f.write(f" start_year                          = {self.start_date_dt.year},\n")
            f.write(f" start_month                         = {self.start_date_dt.month},\n")
            f.write(f" start_day                           = {self.start_date_dt.day},\n")
            f.write(f" start_hour                          = {self.start_date_dt.hour},\n")
            f.write(f" start_minute                        = {self.start_date_dt.minute},\n")
            f.write(f" start_second                        = {self.start_date_dt.second},\n")
            f.write(f" end_year                            = {self.end_date_dt.year},\n")
            f.write(f" end_month                           = {self.end_date_dt.month},\n")
            f.write(f" end_day                             = {self.end_date_dt.day},\n")
            f.write(f" end_hour                            = {self.end_date_dt.hour},\n")
            f.write(f" end_minute                          = {self.end_date_dt.minute},\n")
            f.write(f" end_second                          = {self.end_date_dt.second},\n")
            f.write(f" history_interval_m                  = {self.history_interval_m},\n")
            f.write(f" history_interval_s                  = {self.history_interval_s},\n")
            f.write(f" frames_per_outfile                  = {self.frames_per_outfile},\n")
            f.write("/\n\n")

            f.write("&domains\n")
            f.write(f" west_east_grid_points               = {self.west_east},\n")
            f.write(f" south_north_grid_points             = {self.south_north},\n")
            f.write(f" dx                                  = {self.dx},\n")
            f.write(f" dy                                  = {self.dy},\n")
            f.write("/\n")

In [ ]:
# if __name__ == "__main__":
#     # Example usage
#     wrf_namelist = WRF_Namelist(
#         start_date='2024-07-15-00-00-00',
#         run_time='01-00-00-00',
#         domain_coords={'north': 50, 'south': 40, 'west': -130, 'east': -120},
#         horizontal_resolution=25
#     )
#     wrf_namelist.generate_namelist('namelist.input')